In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import torch
import torch.optim as optim
import torch.nn as nn
from data import get_midi_data, MidiLoader
from handcrafted import HandcraftedKeyModel
from lstmmodel import LSTMModel
import matplotlib.pyplot as plt
from torch.autograd import Variable
import gc

batch_size = 1
handcrafted = HandcraftedKeyModel()
midi_data = get_midi_data(handcrafted)
midi_loader = MidiLoader(midi_data, batch_size=batch_size, shuffle=True)

In [3]:
net = LSTMModel(129, 50, 50, 2)
net.cuda()

LSTMModel (
  (dropout): Dropout (p = 0.5)
  (embedding): Embedding(129, 50)
  (rnn): LSTM(50, 50, num_layers=2, dropout=0.5)
  (linear): Linear (50 -> 84)
)

In [4]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=.9)

In [5]:
def reset_hidden(hidden):
    return (Variable(hidden[0].data), Variable(hidden[1].data))

for epoch in xrange(2):
    running_loss = 0.0
    hidden = net.init_hidden(batch_size)
    for i, data in enumerate(midi_loader):
        inputs, labels = data
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        hidden = reset_hidden(hidden)
        optimizer.zero_grad()
        output, hidden = net(inputs, hidden)
        loss = criterion(output.view(-1, 7*12), labels.view(-1, 7*12))
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
        gc.collect()
        if i % 2000 == 1999:
            print '[{}, {}] loss: {}'.format(epoch + 1, i + 1, running_loss / 2000)
            running_loss = 0.0
print 'finished training'

/usr/local/lib/python2.7/dist-packages/midi-0.2.3-py2.7.egg/midi/fileio.py:66: Warning: Unknown Meta MIDI Event: 9


CuDNNError: 3: CUDNN_STATUS_BAD_PARAM